In [1]:
import timbermafia
import logging
import sys
lorem = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."

In [4]:
timbermafia.configure(columns=120, bold=False, palette='sensible',
                      enclose=False, show_separator=False,
                      format='{asctime} | {levelname} | {name}.{funcName} | {message}')
timbermafia.add_handler(stream=sys.stdout, filename='/tmp/testlog.log',)

log = logging.getLogger('testing')
def test_log_with_a_very_long_name():
    log.debug('test from func')

TypeError: __init__() missing 1 required positional argument: 'filename'

In [ ]:
log.header('Timbermafia testing')
log.info(lorem)
log.info('This looks like a filename: /tmp/a_file.db')
log.info('This looks like a url: https://someaddress.com')

log.debug('a debug message')
log.warning('a warning message')
log.error('found error fdsjkabgfsdjkabngjksdbagnasngjkasngjkasdngjknsd we can recover from')
log.fatal('encountered a fatal error')
log.info('Message containing separators like | @ $ etc are unaffected in messages')
test_log_with_a_very_long_name()
print(lorem)

In [ ]:
f = log.parent.handlers[0].formatter
f._fmt
f.separator

In [ ]:
l = logging.LogRecord('name', logging.DEBUG, '', 21, 'my title', [], False, func='my_func')
l.getMessage()
l.created